## Import function libraries

In [1]:
import numpy as np
import pandas as pd
import scipy
import scipy.stats as stats
import scipy.io as sio
import matplotlib.pyplot as plt
import nibabel as nib

import os
from os.path import join, exists, split
from pathlib import Path
import sys
import time
import urllib.request
import copy
import warnings
from tqdm import tqdm
from pprint import pprint
from bids.layout import BIDSLayout
warnings.filterwarnings('ignore')

import glmsingle
from glmsingle.glmsingle import GLM_single

In [2]:
datadir = Path('/tank/shared/2023/dist_supp_pRF/pilot_data/')
datadir_bids = Path(datadir, 'bids_data')
datadir_fmriprep = Path(datadir, 'derivatives', 'fmriprep')
datadir_freesufer = Path(datadir, 'derivatives', 'freesurfer')
outputdir = Path(datadir, 'derivatives', 'GLMsingle')

print(f'directory of dataset:\n\t{datadir}\n')
print(f'directory to save example1 outputs:\n\t{outputdir}\n')

directory of dataset:
	/tank/shared/2023/dist_supp_pRF/pilot_data

directory to save example1 outputs:
	/tank/shared/2023/dist_supp_pRF/pilot_data/derivatives/GLMsingle



In [3]:
sub = '001'
ses = '01'
run = 3
angles = [*range(0, 360, 15)]
TR = 1.6 #s
pseudo_TR = 0.9
TR_nr = 346
fixation_dur = 0.4
total_time = TR_nr*TR

## Load files

In [27]:
bids_layout = BIDSLayout(datadir_bids, validate=False)
fmriprep_layout = BIDSLayout(datadir_fmriprep, validate=False)

# event_file
event_file = bids_layout.get(subject=sub, 
                             session=ses, 
                             task='ping', 
                             run=3, 
                             suffix="events",
                             extension='tsv')[0]
print(event_file.path)
ev_df = pd.read_csv(event_file.path, sep='\t')

# cifti file
cifti_file = fmriprep_layout.get(subject=sub, 
                             session=ses, 
                             task='ping', 
                             run=3, 
                             suffix="bold",
                             extension='dtseries.nii')[0]
print(cifti_file.path)
datvol = nib.load(cifti_file)
at = np.asanyarray(datvol.dataobj)
print(at.shape)

/tank/shared/2023/dist_supp_pRF/pilot_data/bids_data/sub-001/ses-01/func/sub-001_ses-01_task-ping_run-03_events.tsv
/tank/shared/2023/dist_supp_pRF/pilot_data/derivatives/fmriprep/sub-001/ses-01/func/sub-001_ses-01_task-ping_run-3_space-fsLR_den-170k_bold.dtseries.nii
(608, 170494)


## Transfer the design matrix

In [5]:
start_timepoint = ev_df[(ev_df['trial_nr']==1) & 
                        (ev_df['event_type']=='pulse')]['onset'].values[0]

# Correct the start time
ev_df['onset'] = ev_df['onset'] - start_timepoint
onset_df = ev_df[['onset', 'angle_Ping', 'ori_Ping', 'direction']][(ev_df['trial_type']=='PingTrial') &
      (ev_df['event_type']=='stimulus')].reset_index(drop=True)

In [6]:
onset_df['onset'] = onset_df['onset']*10
onset_df['onset'] = onset_df['onset'].astype('int')
onset_df['angle_Ping'] = onset_df['angle_Ping'].astype('int')
onset_df['ori_Ping'] = onset_df['ori_Ping'].astype('int')

In [7]:
dm = np.zeros((len(np.arange(0, total_time, pseudo_TR)), len(angles)), dtype=int)
for index, row in onset_df.iterrows():
    TR_ind = int((row['onset'] - fixation_dur*10)/(pseudo_TR*10))
    column_ind = angles.index(row['angle_Ping'])
    dm[TR_ind, column_ind] = 1

In [8]:
df_dm = pd.DataFrame(dm, columns=angles)

## Do GLMsingle